In [ ]:
import json

#create the docs_raw variable from the .json file
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []


#siccome elastic search vuole tutti i documenti su un livello, mettiamo ad ogni documento il suo corso
#prima i documenti erano suddivisi in sezione, in base al corso
#guarda le differenze tra docs_raw e documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents

In [ ]:
pip install sentence_transformers==2.7.0

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
#this model has 768 length
model = SentenceTransformer("all-mpnet-base-v2")

In [ ]:
model.encode("this is a simple sentence")

In [ ]:
operations=[]

for doc in documents:
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [ ]:
pip install elasticsearch

In [ ]:
#initiate the elastic search connection
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()


In [ ]:
#creating a mapping (aka index_settings). It defines how a document is stored and indexed
index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"},
                #tipo -> dense_vector, dimensione -> 768, la metrica di similarità che andrà ad usare
                "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
            }
        }
    }


In [ ]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


In [ ]:
#add documents into index
import tqdm
#dont run this cell several times, or (idk why) some doc duplicated. If so, try to delete the index ->
#es_client.indices.delete(index=index_name)
from tqdm.auto import tqdm
for doc in tqdm(documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)
    
    

In [ ]:
#create a query

search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [ ]:
#il campo "field" serve per dire al VDB di andare a cercare nel suddetto campo
#il campo "k" serve per cercare i k vettori più vicini alla query
#il campo "num_candidates" serve per indicare in quanti documenti andrà fatta la ricerca

query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [ ]:
#let's search into VDB
#il campo "knn" serve per indicare 
#il campo "source" serve per indicare quali campi si vuole nella risposta

res = es_client.search(index=index_name, knn=query, source=["text","section","question","course"])
res['hits']['hits']

In [ ]:
#per fare una corretta ricerca semantica, dobbiamo andare a trasformare la nostra query in un vettore
#quando facciamo una ricerca normale(non semantica), i risultati avranno uno score compreso tra 0 e 1


knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k":5,
    "num_candidates": 10000
}

#il campo "explain" serve per avere più informazioni su come lo score è calcolato
#si può creare una propria scoring function
response = es_client.search(
    index=index_name,
    query={
        "match":{
            "course": "data-engineering-zoomcamp"
        },
    },
    knn=knn_query,
    source=["text","section","question","course"],
    size=5,
    explain=True
)

response['hits']['hits']

In [ ]:
#cella per vedere quanti documenti sono presenti nel VDB

search_query = {
    "size": 10000,
    "query": {
        "match_all": {}
    }
        }
    

response = es_client.search(index=index_name, body=search_query)
len(response['hits']['hits'])